In [1]:
# Initial Imports
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from pathlib import Path
import regex as re
import nltk
from datetime import date
import twint
import nest_asyncio
nest_asyncio.apply()
nltk.download('stopwords')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mike4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Initialize set of inputs
# List of Stocks
stocks = ['microsoft','sony','pfizer','regeneron','moderna']

# Set up Date range
import datetime
from dateutil.relativedelta import relativedelta

today = datetime.date.today()
end_time = today.strftime('%Y-%m-%d %H:%M:%S')
start_time = today - relativedelta(years=1)
start_time = start_time.strftime('%Y-%m-%d %H:%M:%S')
print(start_time)

2020-01-19 00:00:00


In [ ]:
# Define data_crawlers function
stock_df = pd.DataFrame()
df = pd.DataFrame()

In [4]:
def data_crawlers(stock):
    c = twint.Config()
    c.Search = stock
    c.Hide_output = True
    c.Limit = 1000 
    #c.Store_csv = True
    c.Since = start_time
    c.Until = end_time
    #c.Output = f"{stock}.csv"
    c.Pandas = True
    c.Pandas_au = True
    twint.run.Search(c)
    #df = twint.storage.panda.Tweets_df

In [5]:
# Deploy data_crawlers function on stocks list
for i in stocks: 
    data_crawlers(i)
    df = twint.storage.panda.Tweets_df
    stock_df = stock_df.append(df)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 125.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 216.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 343.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 512.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 729.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataEx

In [6]:
stock_df = stock_df.reset_index(drop=True)
stock_df.tail()

""


In [7]:
df.head()

""


In [8]:
# Define function to read in csv
from pathlib import Path
def csv_import(stock):
    stock_path = Path(f'{stock}.csv')
    return stock_path

In [9]:
path_list = []
for i in stocks:
    i_df = pd.DataFrame()
    path_list.append(csv_import(i))
    for x in path_list: 
        x_df = pd.DataFrame()
        i_df = pd.read_csv(x)

FileNotFoundError: [Errno 2] No such file or directory: 'microsoft.csv'

In [ ]:
i_df

In [ ]:
x_df.tail()

In [ ]:
csv_import('microsoft')